# ECS713: week 5 Lab Sheet

This lab sheet covers 
- typeclasses

## Learning Objectives

By the time you complete this sheandet you should be able to
- understand how to declare a new typeclass
- how to make a type an instance of a typeclass
- recognise some common typeclasses and the functions they provide
- understand the use of the deriving connective

## Turn off the annoying linter

Run the cell below to turn off the annoying linter, which suggests improvements to your code that aren't appropriate for these exercises. 

In [1]:
:opt no-lint

## Task 1. Basic Type Classes

Type classes are Haskell's equivalent of interfaces in Java. They are a way of specifying that a type (class in Java) should have certain operations. Very often the operations should have some intuitive meaning, but the compiler has no way of enforcing that. 

Example:
This is a particularly stupid example. A type `a` is an instance of the typeclass `Cloop`if it has three operations, 
`moop`, `boop` and `poop`, with types as below. 

In [3]:
class Cloop a where 
  moop :: a -> a
  boop :: a -> a -> a
  poop :: a -> Bool

The Haskell typechecker understands that, after this declaration, if you use functions called `moop`, `boop` or `poop`, then they'd better be on something of type class `Cloop`.

Test:

In [4]:
stoop a = boop (moop a) a
:t stoop

stoop :: forall a. Cloop a => a -> a

Notice the `Cloop a` there. That is what says `a` had better be of type class `Cloop`. 

So let's give Int the appropriate functions:

In [5]:
moop :: Int -> Int
moop n = n+1
boop :: Int -> Int -> Int
boop m n = m+n
poop :: Int -> Bool
poop m = m==0

In [8]:
it = stoop 4 :: Int

: 

If you've run the appropriate cells here you should get a complaint: 
``<interactive>:1:1: error:
    • No instance for (Cloop Int) arising from a use of ‘stoop’
    • In the expression: stoop 4 :: Int
      In an equation for ‘it’: it = stoop 4 :: Int``

It's not enough to have the functions, you have to explicitly explain that `Int` is supposed to be a member of the typeclass: 

In [9]:
instance Cloop Int where
  moop n = n+1
  boop m n = m+n
  poop m = m==0

In [10]:
stoop 4::Int

9

Exercise: Make `Char` and `Bool` members of the typeclass `Cloop`. 

In [11]:
instance Cloop Bool where
  moop x = not x
  boop x y = x || y
  poop x = x

instance Cloop Char where
  moop x = x
  boop x y = x
  poop x = x == 'a'

Warning: this will not work with complex types like `String` (i.e. ones that are constructed from simpler types). If you try to make `String` an instance of a typeclass, you will either get told that you can't use a synonym, or if you use `[Char]`, that you can only have a constructor applied to a type variable. Here is one way round it. Use a wrapper to convert an open complex type, into a closed simple one. 

In [12]:
data PackString = Pack String
  deriving Show

instance Cloop PackString where
  moop n = n
  boop m n = m
  poop m = True
stoop $ Pack "a"

Pack "a"

## Task 2. Inbuilt type classes. 

Some type classes come wth the system. Each has some basic functions. Use the documentation at: https://hackage.haskell.org/package/base-4.14.0.0/docs/Prelude.html#g:4
and at
https://hackage.haskell.org/package/base-4.14.0.0/docs/Prelude.html#g:22
to find out which functions you need for each of the following type classes: 

`Eq` requires

In [14]:
-- For Eq you need to implement either (==) or (/=)

data Test = T1 | T2

instance Eq Test where
   T1 == T1 = True
   T1 == T2 = False
   T2 == T1 = False
   T2 == T2 = True

T2 == T1

False

`Ord` requires

In [15]:
instance Ord Test where
   T1 <= T1 = True
   T1 <= T2 = True
   T2 <= T2 = True
   T2 <= T1 = False

T1 <= T2

True

`Show` requires

In [16]:
instance Show Test where
   show T1 = "T1 here"
   show T2 = "T2 there"
   
print T1

T1 here

`Read` requires

In [19]:
-- Minimal complete definition readsPrec | readPrec

`Eq` is the class of types supporting equality. 

`Ord` is the class of types supporting an order. 

`Show` is the class of types whose values can be printed (converted to `String`s). 

`Read` is the class of types whose values can be parsed from `String`s.

We've seen that the compiler will take any functions of the appropriate type to make instances of the typeclass. 
But often there are formal requirements couched in terms of expected equations, and informal requirements in terms 
of expectations. For simple datastructures, the compiler can derive a default instance of the standard typeclasses. 
It is instructed to do this using the `deriving` keyword. 

**EXERCISE** Here are two definitions of the Booleans, both using an enumerated type, and both deriving membership of typeclasses. What is the difference (and more importantly, how would it impact programs)? `T1` and `T2` represent `True`, and `F1` and `F2` represent `False`.

In [20]:
data B1 = F1 | T1 deriving (Eq, Ord, Show, Read) 

In [21]:
data B2  = T2 | F2 deriving (Eq, Ord, Show, Read) 

Explain difference here:

The only difference is the order in which True and False are given. In the first case F1 < T1 (so True is less than False), while in the second True is less than False.

## Task 3. Hierarchies

Sometimes we require that when we want to make a type a member of a typeclass, it should already be a member of some other. There is a hierarchy. 

Example: `Ord` requires that the type already be a member of `Eq`. 

```class Eq a => Ord a where ...```

**Exercise:** Make `PackString` an instance of `Eq` where two packed strings are regarded as equal if they have the same length. 

In [22]:
instance Eq PackString where
   Pack s1 == Pack s2 = length s1 == length s2
   
Pack "hi" == Pack "oi"

True

**Exercise:** Make `PackString` an instance of `Ord` where the ordering is that one string is shorter than the other. 

In [24]:
instance Ord PackString where
   Pack s1 <= Pack s2 = length s1 <= length s2
   
Pack "hi" <= Pack "hello"

True

## Task 4. Deriving Show

**Exercise:** `PackStringB` is defined similarly to `PackString` but without deriving `Show`. Can you explain why you get an error when trying to evaluate `PackB "aaa"` but not `unpackB (packB "aaa")`?

In [25]:
data PackStringB = PackB String
unpackB (PackB s) = s

In [28]:
PackB "aaa"

: 

In [27]:
unpackB (PackB "aaa")

"aaa"

Explanation goes here: In the first Haskell does not know yet show to "show" an expression of type PackStringB, we need to make PackStringB an instance of the Show typeclass. In the second case we get a String "aaa", and Haskell already knows how to "show" a String.

Try making PackStringB an instance of Ord **without** also giving code to make it an instance of Eq. Is this possible?

In [29]:
instance Ord PackStringB where
   PackB s1 <= PackB s2 = length s1 <= length s2

: 

Explanation: Explain the issue here: Ord is a sub-class of Eq. So, in order to make PackStringB an instance of Ord, we first need to make it a member of the type class Eq.